In [1]:
from weiner_variation.sim.data_structures import DrawDurations
from weiner_variation.sim.process import PASS_SEQUENCE, DIAMETER, TEMPERATURE, create_in_profile, REVERSING_PAUSE_DURATION
from weiner_variation.sim.config import SAMPLE_COUNT, FIELDS, SEED
from weiner_variation.config import DATA_DIR

import pandas as pd
import numpy as np
from scipy.stats import norm, gamma
from copy import deepcopy
import tqdm
from multiprocessing import Pool

/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (depth - l23(_alpha) * np.sin(_alpha) - fh(_alpha)) / (1 - np.cos(_alpha))
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:83: RuntimeWarning: invalid value encountered in scalar multiply
  width / 2 - _r2 * np.sin(_alpha) - l23(_alpha) * np.cos(_alpha)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:84: RuntimeWarning: invalid value encountered in scalar multiply
  - fw(_alpha) - (_r2 + r4) * np.sin(_alpha4)
/home/maxwe/Git/publication-weiner-variation/.venv_sim/lib/python3.10/site-packages/pyroll/core/grooves/generic_elongation_solvers.py:79: RuntimeWarning: divide by zero encountered in scalar divide
  _r2 = (de

In [2]:
import pyroll.basic as pr

In [3]:
DIAMETER_STD = 1e-3
TEMPERATURE_STD = 10
OUTPUT_FILENAME = "sim_durations_results.csv"

In [4]:
df_durations = pd.read_csv(DATA_DIR / "duo_pauses_dist.csv", header=0, index_col=0)
df_durations

,mean,std,max,min,loc,alpha,beta
R1-R2,6.633088,1.471388,11.047252,5.115,5.425000,1.248549,1.070947
R2-R3,4.435441,0.995052,7.420597,3.040,3.002936,6.802674,0.198825
R3-R4,5.988015,2.192780,12.566355,4.015,4.507276,1.823817,0.733746
R4-R5,5.654853,0.470526,7.066431,4.745,4.641153,4.547398,0.232675
R5-R6,5.044559,0.566902,6.745266,3.635,3.635000,6.605120,0.231032
R6-R7,5.195588,0.588969,6.962496,4.235,4.235579,6.937555,0.125272
R7-R8,6.435882,1.557230,11.107573,4.705,5.160270,1.637811,0.746130
R8-R9,6.080735,0.723092,8.250011,4.745,4.724997,3.723512,0.323734
R9-R10,7.008235,0.775194,9.333816,5.260,5.271222,6.443636,0.286210
R10-F1,9.233309,0.399014,10.430352,8.290,8.260580,12.416743,0.074943


In [5]:
diameter_dist = norm(loc=DIAMETER, scale=DIAMETER_STD)
temperature_dist = norm(loc=TEMPERATURE, scale=TEMPERATURE_STD)
durations_dist = gamma(a=df_durations["alpha"]["all"], scale=df_durations["beta"]["all"], loc=df_durations["loc"]["all"])
durations_dist_last = gamma(a=df_durations["alpha"]["R10-F1"], scale=df_durations["beta"]["R10-F1"], loc=df_durations["loc"]["R10-F1"])

In [6]:
RNG = np.random.default_rng(SEED)
diameters = diameter_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
temperatures = temperature_dist.rvs(random_state=RNG, size=SAMPLE_COUNT)
durations = np.concatenate([
    durations_dist.rvs(random_state=RNG, size=(SAMPLE_COUNT, 9)), 
    durations_dist_last.rvs(random_state=RNG, size=(SAMPLE_COUNT, 1))
], axis=1)

draws = [
    DrawDurations(d, t, dur)
    for d, t, dur in zip(diameters, temperatures, durations)
]

In [7]:
def worker(draw: DrawDurations):    
    ip = create_in_profile(
            diameter=draw.diameter,
            temperature=draw.temperature,
    )

    sequence = deepcopy(PASS_SEQUENCE)  
    
    transports = [u for u in sequence if isinstance(u, pr.Transport)]
    for t, d in zip(transports, draw.durations):
        t.duration = d
    
    sequence.solve(ip)       

    return (
        draw.__dict__.items() |> map$(t -> (("draw", t[0]), t[1])) |> dict
    ) | (
        FIELDS.items()
        |> starmap$((key, extractor) -> (
                sequence.units 
                |> filter$(u -> isinstance(u, pr.RollPass)) 
                |> map$(u -> ((key, u.label), extractor(u))) 
                |> filter$(t -> t[1] is not None)
            )
        )
        |> flatten
        |> dict            
    )

In [8]:
results = tqdm.tqdm(Pool().imap(worker, draws), total=SAMPLE_COUNT) |> list

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:02<00:00, 39.69it/s]


In [9]:
df = results |> filter$(e -> "error" not in e) |> enumerate |> dict |> pd.DataFrame |> .T |> .infer_objects()
df

draw                                                                  \
    diameter  temperature                                          durations   
0   0.050390  1424.351002  [5.5265345122075304, 6.537977750617033, 5.1076...   
1   0.050772  1429.997863  [4.502209403606896, 7.649448908346107, 6.73243...   
2   0.051325  1412.349194  [5.143444777223476, 6.758532761690237, 4.51409...   
3   0.049996  1409.882516  [7.326346171735677, 4.756085164927763, 4.78279...   
4   0.050787  1423.286940  [5.707022295482241, 7.515212833741915, 5.73053...   
..       ...          ...                                                ...   
95  0.049713  1400.051662  [7.093084502631746, 5.163733116741946, 5.10497...   
96  0.050259  1443.972998  [4.998620815794158, 6.166850711934137, 5.82800...   
97  0.049412  1416.344465  [4.352547936729915, 4.274033902932483, 6.12483...   
98  0.049200  1430.227803  [5.953224829203364, 5.113201987822091, 5.27450...   
99  0.049632  1419.146461  [5.773225010406285, 6.272090619947709, 6.55801...   

       roll_force                                                              \
               R1             R2             R3             R4             R5   
0   398347.173717  229383.275649  425029.463532  179210.352764  252246.061630   
1   401235.374067  228122.939247  423352.804874  181224.251463  257190.546444   
2   428907.045641  242275.626252  437243.961465  181000.988365  259866.862455   
3   402869.460931  235870.164176  433032.768767  182206.598240  255459.503358   
4   407514.948649  233608.696666  431245.487748  182102.276251  259660.870123   
..            ...            ...            ...            ...            ...   
95  405604.597769  237393.107206  439581.157429  185787.164885  267260.611415   
96  378858.769842  218569.714919  408010.630522  174554.295599  244666.011066   
97  385015.453667  222358.607339  415612.884257  179381.408183  251181.591025   
98  369094.577205  217313.476843  412258.288064  176778.021343  249570.751585   
99  387146.852030  225590.894763  425211.966270  183459.388043  257019.453563   

                                  ...    strain                                \
               R6             R7  ...        R5        R6        R7        R8   
0   139028.623489  258566.220705  ...  0.596759  0.499470  0.728197  0.596492   
1   139680.207442  260620.298104  ...  0.596871  0.499533  0.728224  0.596532   
2   141326.925500  261696.087521  ...  0.597029  0.499600  0.728252  0.596536   
3   137801.374637  265066.730183  ...  0.596684  0.499460  0.728205  0.596561   
4   139409.828971  270376.157030  ...  0.596896  0.499552  0.728252  0.596620   
..            ...            ...  ...       ...       ...       ...       ...   
95  143251.535525  264713.922514  ...  0.596657  0.499527  0.728241  0.596567   
96  131773.319925  249217.204374  ...  0.596665  0.499382  0.728127  0.596401   
97  135522.365826  252912.030459  ...  0.596471  0.499365  0.728141  0.596433   
98  133751.137495  255925.204892  ...  0.596390  0.499327  0.728130  0.596444   
99  136645.584437  255178.646185  ...  0.596576  0.499431  0.728170  0.596453   

                                                                
          R9       R10        F1        F2        F3        F4  
0   0.458514  0.459743  0.342630  0.346555  0.447869  0.416458  
1   0.458566  0.459890  0.342755  0.346752  0.447964  0.416633  
2   0.458545  0.459804  0.342668  0.346606  0.447893  0.416507  
3   0.458584  0.459895  0.342727  0.346683  0.447928  0.416558  
4   0.458610  0.459885  0.342706  0.346630  0.447900  0.416518  
..       ...       ...       ...       ...       ...       ...  
95  0.458570  0.459825  0.342679  0.346615  0.447896  0.416512  
96  0.458494  0.459816  0.342687  0.346642  0.447910  0.416543  
97  0.458495  0.459757  0.342643  0.346592  0.447888  0.416500  
98  0.458485  0.459730  0.342644  0.346609  0.447898  0.416521  
99  0.458489  0.459732  0.342648  0.346608  0.447897  0.416518  

[100 rows x 1

In [10]:
df.to_csv(DATA_DIR / OUTPUT_FILENAME)